In [ ]:
import os
from dotenv import load_dotenv

from langchain.agents import AgentType, Tool, initialize_agent
# from langchain.chat_models import ChatOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.memory import ConversationBufferMemory

from chatrag.retriever import create_retrieval_chain, create_retriever_from_csv

In [ ]:
# import logging
# logging.basicConfig(level=logging.DEBUG)

In [ ]:
os.environ["LANGCHAIN_WANDB_TRACING"] = "true"
os.environ["WANDB_PROJECT"] = "chatrag"
os.environ["WANDB_NOTEBOOK_NAME"] = "retriever_agent_react_poc_azure"

In [ ]:
load_dotenv()

In [ ]:
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
os.environ["OPENAI_API_BASE"] = os.getenv("AZURE_OPENAI_ENDPOINT", "")
os.environ["OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_KEY", "")

## Data preparation

In [ ]:
retriever_llm = AzureChatOpenAI(
    temperature=0,
    model="gpt-35-turbo-16k",
    max_tokens=2000,
    deployment_name="gpt-35-test"
)
retriever = create_retriever_from_csv(
    csv_path="../data/movies_title_overview_vote.csv",
    metadata_columns_dtypes={"vote_average": "float"},
    llm=retriever_llm,
    embedding_deployment="embedding-test"
)

In [ ]:
# retriever.get_relevant_documents("I want to watch a movie about space battles.")

# Vectorstore DB

In [ ]:
retrieval_chain_llm = AzureChatOpenAI(
    temperature=0,
    model="gpt-35-turbo-16k",
    max_tokens=2000,
    deployment_name="gpt-35-test"
)
g_media_sq_retrieval_chain = create_retrieval_chain(retriever=retriever, llm=retrieval_chain_llm)

In [ ]:
# print(g_media_sq_retrieval_chain.run("I want to watch a movie about space battles."))

# Tooling

In [ ]:
retriever_description = """Movie search tool. The action input must be just topics in a natural language sentence."""

In [ ]:
tools = [
    Tool(
        name = "Search movies",
        func=g_media_sq_retrieval_chain.run,
        description=retriever_description
    )
]

# Prompt templates

In [ ]:
# flake8: noqa
PREFIX = """You are Diverger Assistant, an language model created by OpenAI and used by Diverger, an AI company. You are an assistant expert in movie recommendation.

Take a deep breath, use the TOOLS when needed and ALWAYS follow RESPONSE FORMAT INSTRUCTIONS."""

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

agent_kwargs = {
        "system_message": PREFIX,
    }

# Chains

In [ ]:
agent_llm_model = AzureChatOpenAI(
    temperature=0,
    model="pruebas-azure-gpt4",
    max_tokens=2000,
    deployment_name="gpt-4"
)

In [ ]:
general_chat_agent = initialize_agent(
    tools=tools,
    llm=agent_llm_model,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
    agent_kwargs=agent_kwargs,
    handle_parsing_errors=True
)

# TESTS

### Run

In [ ]:
from langchain.globals import set_debug
set_debug(True)

In [ ]:
response = general_chat_agent.run(input="Hi, my name is Aitor")
print("\nA partir de aquí la respuesta:\n")
print(response)

In [ ]:
response = general_chat_agent.run(input="""I would like to watch a movie""")
print("\nA partir de aquí la respuesta:\n")
print(response)

In [ ]:
response = general_chat_agent.run(input="""I want to watch a movie about outer space exploration.""")
print("\nA partir de aquí la respuesta:\n")
print(response)

In [ ]:
response = general_chat_agent.run(input="I've changed my mind. Would you suggest me a good genre for just relaxing?.")
print("\nA partir de aquí la respuesta:\n")
print(response)

In [ ]:
response = general_chat_agent.run(input="Sounds good, give me a movie about it.")
print("\nA partir de aquí la respuesta:\n")
print(response)

In [ ]:
response = general_chat_agent.run(input="By the way, what's your name?.")
print("\nA partir de aquí la respuesta:\n")
print(response)